In [1]:
import nltk
import numpy as py
import pandas as pd
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer 
lemmatizer = WordNetLemmatizer()

In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [3]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

In [4]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mojim\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mojim\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mojim\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [7]:
df=pd.read_excel("Case Management Note_20200113.xlsx")

In [8]:
dsplit=pd.read_excel('merge_without_YLS_3152.xlsx')

In [12]:
def group(data,column):
    gp = data.groupby(column)    
    result=[gp.get_group(x) for x in gp.groups]
    return result
new=group(dsplit,'YES Transition')
print(new[0]['YES Transition'])

non_par=new[0]
par=new[1]
npname=list(non_par['MID'])
pname=list(par['MID'])


0       No
1       No
2       No
3       No
4       No
        ..
3145    No
3146    No
3147    No
3150    No
3151    No
Name: YES Transition, Length: 1824, dtype: object


In [13]:
have=list(df['MID'])
nd=list(dsplit['MID'])
noinfo=[] #no info about participant
for i in have:
    if i not in nd:
        if i not in noinfo:
            noinfo.append(i)
print(noinfo)


[504256, 504337, 504626, 504765, 505223, 507631, 372399, 373636, 501395, 501786, 501900, 501925, 502045, 502444, 502636, 502756, 503017]


In [15]:
paraddon=[]
for i in have:
    if i in pname:
        paraddon.append('Yes')
    elif i in npname:
        paraddon.append('No')
    else:
        paraddon.append('No Info')
print(len(paraddon))

317473


In [36]:
df['YES Transition']=paraddon
df.head()

,MID,DATE_OF_NOTE,ADMIN_ENTRY,CONTACT_TYPE,CONTACT_CONNECTION,HOW_WAS_CONTACT_MADE,WHO_WAS_CONTACTED,WHY,COMMENTS,COMMENTS_CLOB,...,STAFF_WHO_PH_OTH,STAFF_WHO_PH_EXT,STAFF_WHO_PH_EXT_OTH,STAFF_WHO_PH_INT,STAFF_WHO_PH_INT_OTH,STAFF_NOTES,STAFF_REC,CMN_COMMENTS,CMN_STAFFING_TYPE,YES Transition
0,500554,09-JUN-15,No,Electronic Communications,Contact Made,Telephone,Client,"Behavior Management, Check-in, Health Services...",CW spoke to Angel today and explained that CW ...,CW spoke to Angel today and explained that CW ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CW spoke to Angel today and explained that CW ...,NaN,No
1,500554,18-JUN-15,No,Electronic Communications,Contact Made,Telephone,"Client, Internal Collateral",Behavior Management,CW spoke to clinician Katlyn today who reports...,CW spoke to clinician Katlyn today who reports...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CW spoke to clinician Katlyn today who reports...,NaN,No
2,500554,11-MAY-15,No,Electronic Communications,Contact Made,Telephone,Internal Collateral,"Behavior Management, Check-in",CW spoke to Laura today and asked how Angel wa...,CW spoke to Laura today and asked how Angel wa...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CW spoke to Laura today and asked how Angel wa...,NaN,No
3,500554,07-MAY-15,No,Program visit,Contact Made,Face-to-face,"Client, Internal Collateral","Behavior Management, Check-in, Educational/Voc...",CW met with Angel and Laura today for Angel's ...,CW met with Angel and Laura today for Angel's ...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CW met with Angel and Laura today for Angel's ...,NaN,No
4,500554,13-MAY-15,No,Electronic Communications,Contact Made,Telephone,Client,"Behavior Management, Check-in, Educational/Voc...",CW spoke to Angel today and he reports he is a...,CW spoke to Angel today and he reports he is a...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,CW spoke to Angel today and he reports he is a...,NaN,No


In [42]:
gp=group(df,'YES Transition')
print(gp[0]['YES Transition'],gp[1]['YES Transition'],gp[2]['YES Transition'])
dfnp=gp[0]
dfp=gp[2]

dfnp_id=list(dfnp['MID'])
dfp_id=list(dfp['MID'])


0         No
1         No
2         No
3         No
4         No
          ..
317468    No
317469    No
317470    No
317471    No
317472    No
Name: YES Transition, Length: 122555, dtype: object 50519     No Info
50520     No Info
50521     No Info
50522     No Info
50523     No Info
           ...   
314109    No Info
314110    No Info
314111    No Info
314112    No Info
314113    No Info
Name: YES Transition, Length: 2093, dtype: object 1309      Yes
1310      Yes
1311      Yes
1312      Yes
1313      Yes
         ... 
316974    Yes
316975    Yes
316976    Yes
316977    Yes
316978    Yes
Name: YES Transition, Length: 192825, dtype: object
<class 'pandas.core.frame.DataFrame'>


In [44]:
np_race=group(non_par,'Race (2018 Std.)')
p_race=group(par,'Race (2018 Std.)')

#black or African American
np_black=np_race[1]
p_black=p_race[2]

npb=list(np_black['MID'])
pb=list(p_black['MID'])

#Hispanic
np_hispanic=np_race[3]
p_hispanic=p_race[4]

nph=list(np_hispanic['MID'])
ph=list(p_hispanic['MID'])

#white
np_white=np_race[8]
p_white=p_race[8]

npw=list(np_white['MID'])
pw=list(p_white['MID'])

racenp=[]
for i in dfnp_id:
    if i in npb:
        racenp.append('Black')
    elif i in nph:
        racenp.append('Hispanic')
    elif i in npw:
        racenp.append('White')
    else:
        racenp.append('No Info')
print(len(dfnp),len(racenp))
dfnp['Race']=racenp

racep=[]
for i in dfp_id:
    if i in pb:
        racep.append('Black')
    elif i in ph:
        racep.append('Hispanic')
    elif i in pw:
        racep.append('White')
    else:
        racep.append('No Info')
print(len(dfp),len(racep))

dfp['Race']=racep
dfp.head()

122555 122555


C:\Users\mojim\anaconda3\lib\site-packages\ipykernel_launcher.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


192825 192825


C:\Users\mojim\anaconda3\lib\site-packages\ipykernel_launcher.py:50: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,MID,DATE_OF_NOTE,ADMIN_ENTRY,CONTACT_TYPE,CONTACT_CONNECTION,HOW_WAS_CONTACT_MADE,WHO_WAS_CONTACTED,WHY,COMMENTS,COMMENTS_CLOB,...,STAFF_WHO_PH_EXT,STAFF_WHO_PH_EXT_OTH,STAFF_WHO_PH_INT,STAFF_WHO_PH_INT_OTH,STAFF_NOTES,STAFF_REC,CMN_COMMENTS,CMN_STAFFING_TYPE,YES Transition,Race
1309,503091,03-JUN-15,No,Home visit,Contact Made,Face-to-face,"Client, Family Member(s)","Behavior Management, Check-in, Educational/Voc...",I met with client today and informed him that ...,I met with client today and informed him that ...,...,NaN,NaN,NaN,NaN,NaN,NaN,I met with client today and informed him that ...,NaN,Yes,White
1310,503091,11-JUN-15,No,Electronic Communications,Contact Attempted,NaN,NaN,NaN,"Tried to reach Joe via phone, no answer. Left ...","Tried to reach Joe via phone, no answer. Left ...",...,NaN,NaN,NaN,NaN,NaN,NaN,"Tried to reach Joe via phone, no answer. Left ...",NaN,Yes,White
1311,503091,16-JUN-15,No,Electronic Communications,Contact Made,Telephone,Parent/Guardian,Check-in,Spoke with Joe's mother Regina over the phone....,Spoke with Joe's mother Regina over the phone....,...,NaN,NaN,NaN,NaN,NaN,NaN,Spoke with Joe's mother Regina over the phone....,NaN,Yes,White
1312,503091,18-JUN-15,No,Electronic Communications,Contact Made,Telephone,Client,"Check-in, Educational/Vocational",Joe contacted me returning a call. Client is i...,Joe contacted me returning a call. Client is i...,...,NaN,NaN,NaN,NaN,NaN,NaN,Joe contacted me returning a call. Client is i...,NaN,Yes,White
1313,503091,10-JUN-15,No,Home visit,Contact Made,Face-to-face,"Client, Family Member(s)","Behavior Management, Educational/Vocational, L...",I went to client's home to check in with him. ...,I went to client's home to check in with him. ...,...,NaN,NaN,NaN,NaN,NaN,NaN,I went to client's home to check in with him. ...,NaN,Yes,White


In [47]:
np_type=group(non_par,'MSO Offense Type')
p_type=group(par,'MSO Offense Type')

#Person    
np_person=np_type[3]
p_person=p_type[2]

npp=list(np_person['MID'])
pp=list(p_person['MID'])

#Weapons
np_weapons=np_type[6]
p_weapons=p_type[5]

npws=list(np_weapons['MID'])
pws=list(p_weapons['MID'])

typenp=[]
for i in dfnp_id:
    if i in npp:
        typenp.append('Person')
    elif i in npws:
        typenp.append('Weapons')
    else:
        typenp.append('No Info')
print(len(dfnp),len(typenp))
dfnp['MSO Offense Type']=typenp

typep=[]
for i in dfp_id:
    if i in pp:
        typep.append('Person')
    elif i in pws:
        typep.append('Weapons')
    else:
        typep.append('No Info')
print(len(dfp),len(typep))
dfp['MSO Offense Type']=typep

dfnp.head()

122555 122555


C:\Users\mojim\anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


192825 192825


C:\Users\mojim\anaconda3\lib\site-packages\ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,MID,DATE_OF_NOTE,ADMIN_ENTRY,CONTACT_TYPE,CONTACT_CONNECTION,HOW_WAS_CONTACT_MADE,WHO_WAS_CONTACTED,WHY,COMMENTS,COMMENTS_CLOB,...,STAFF_WHO_PH_EXT_OTH,STAFF_WHO_PH_INT,STAFF_WHO_PH_INT_OTH,STAFF_NOTES,STAFF_REC,CMN_COMMENTS,CMN_STAFFING_TYPE,YES Transition,Race,MSO Offense Type
0,500554,09-JUN-15,No,Electronic Communications,Contact Made,Telephone,Client,"Behavior Management, Check-in, Health Services...",CW spoke to Angel today and explained that CW ...,CW spoke to Angel today and explained that CW ...,...,NaN,NaN,NaN,NaN,NaN,CW spoke to Angel today and explained that CW ...,NaN,No,Hispanic,Weapons
1,500554,18-JUN-15,No,Electronic Communications,Contact Made,Telephone,"Client, Internal Collateral",Behavior Management,CW spoke to clinician Katlyn today who reports...,CW spoke to clinician Katlyn today who reports...,...,NaN,NaN,NaN,NaN,NaN,CW spoke to clinician Katlyn today who reports...,NaN,No,Hispanic,Weapons
2,500554,11-MAY-15,No,Electronic Communications,Contact Made,Telephone,Internal Collateral,"Behavior Management, Check-in",CW spoke to Laura today and asked how Angel wa...,CW spoke to Laura today and asked how Angel wa...,...,NaN,NaN,NaN,NaN,NaN,CW spoke to Laura today and asked how Angel wa...,NaN,No,Hispanic,Weapons
3,500554,07-MAY-15,No,Program visit,Contact Made,Face-to-face,"Client, Internal Collateral","Behavior Management, Check-in, Educational/Voc...",CW met with Angel and Laura today for Angel's ...,CW met with Angel and Laura today for Angel's ...,...,NaN,NaN,NaN,NaN,NaN,CW met with Angel and Laura today for Angel's ...,NaN,No,Hispanic,Weapons
4,500554,13-MAY-15,No,Electronic Communications,Contact Made,Telephone,Client,"Behavior Management, Check-in, Educational/Voc...",CW spoke to Angel today and he reports he is a...,CW spoke to Angel today and he reports he is a...,...,NaN,NaN,NaN,NaN,NaN,CW spoke to Angel today and he reports he is a...,NaN,No,Hispanic,Weapons


In [52]:
nprace=group(dfnp,'Race')
#print(nprace[0]['Race'],nprace[1]['Race'],nprace[2]['Race'])
npblack=nprace[0]
nphis=nprace[1]
npwhite=nprace[3]

prace=group(dfp,'Race')
#print(prace[0]['Race'],prace[1]['Race'],prace[2]['Race'])
pblack=prace[0]
phis=prace[1]
pwhite=prace[3]

nptype=group(dfnp,'MSO Offense Type')
#print(nptype[0]['MSO Offense Type'],nptype[1]['MSO Offense Type'])
npperson=nptype[1]
npweapon=nptype[2]

ptype=group(dfp,'MSO Offense Type')
#print(ptype[0]['MSO Offense Type'],ptype[1]['MSO Offense Type'])
pperson=ptype[1]
pweapon=ptype[2]

2625      No Info
2626      No Info
2627      No Info
2628      No Info
2629      No Info
           ...   
317264    No Info
317265    No Info
317266    No Info
317267    No Info
317268    No Info
Name: MSO Offense Type, Length: 37064, dtype: object 849       Person
850       Person
851       Person
852       Person
853       Person
           ...  
317468    Person
317469    Person
317470    Person
317471    Person
317472    Person
Name: MSO Offense Type, Length: 66370, dtype: object
1309      No Info
1310      No Info
1311      No Info
1312      No Info
1313      No Info
           ...   
316974    No Info
316975    No Info
316976    No Info
316977    No Info
316978    No Info
Name: MSO Offense Type, Length: 73305, dtype: object 1584      Person
1585      Person
1586      Person
1587      Person
1588      Person
           ...  
316864    Person
316865    Person
316866    Person
316867    Person
316868    Person
Name: MSO Offense Type, Length: 97833, dtype: object


In [62]:
#would, also, know, could, let, thing, may, - , seem, ecc9?, cw, show
addlist=['.',',','!','?','(',')',"'s",'would','also','know','could','let','thing','may','-','seem','ecc','cw',
         'show','go','well','get','take','come','see','make','give','bring','able','regard','date','still','look',
         'new','last','try','reach','morning','weekend','find','night','pm','address','use','hour','yesterday','keep'
        ,'card','put','live','question','like','rrt','respond']
stop_words = set(stopwords.words('english'))
for x in addlist:
    stop_words.add(x)
print(stop_words)

{'whom', 'hadn', 'take', ')', 'mightn', 'don', 'out', "wouldn't", 'be', 'between', 'aren', 'below', 'has', 'which', 'theirs', 'very', 'once', 'doesn', 'this', 'wouldn', '.', "wasn't", 'ourselves', 'until', 'why', 'of', 'hers', 'had', "aren't", 'get', 'such', 'should', 'them', 'shouldn', 's', "doesn't", 'each', 'nor', 'question', 'he', 'show', 'live', 'rrt', 'how', 'shan', "shan't", 'most', 've', "you've", 'yours', 'card', 'same', 'could', 'll', 'on', ',', 'morning', 'no', 'an', 'put', 'after', 'from', 'but', 'both', 'so', 'him', 'your', 'the', 'also', 'yesterday', 'ecc', "won't", 'address', 'where', 'o', 'was', 'more', '(', 'some', 'their', 'those', 'would', 'by', 'if', 'during', "hasn't", 'needn', "you'd", 'who', 'myself', "needn't", 'here', "hadn't", '?', 'pm', 'night', 'when', 'use', "shouldn't", 'about', "it's", 'not', 'they', 'over', 'and', 'won', "haven't", 'did', 'wasn', 'come', 'weekend', 'in', 'regard', 'it', 'does', "'s", 'further', 'these', 'what', 'into', 'y', 'while', 'me'

***dfnp, dfp, npblack, pblack, nphis, phis, npwhite, pwhite, npperson, person, npweapon, pweapon***

In [53]:
datanpb = npblack.COMMENTS.values.tolist()
datapb = pblack.COMMENTS.values.tolist()

datanph = nphis.COMMENTS.values.tolist()
dataph = phis.COMMENTS.values.tolist()

datanpw = npwhite.COMMENTS.values.tolist()
datapw = pwhite.COMMENTS.values.tolist()

datanpp = npperson.COMMENTS.values.tolist()
datapp = pperson.COMMENTS.values.tolist()

datanpwp = npweapon.COMMENTS.values.tolist()
datapwp = pweapon.COMMENTS.values.tolist()

In [54]:
pprint(datanpb[:1])

["This writer attended Daquelle's IEP meeting on yesterday.  Judy Cook, BPS "
 'SESS Coordinator was able to speak with his mom prior to the meeting and was '
 'given permission to hold the meeting.  Mom was not able to attend because '
 'she had to work.  Mom shared her concern that she does not want Daquelle to '
 'return to McKinley.  However, Judy said that he would need to return to '
 'McKinley and the folks at McKinley would need to determine that he no longer '
 'needs that placement.  When Daquelle returns to the community, he would be '
 'in the 12th grade.  \r\n'
 'Daquelle attended the meeting and he asked appropriate questions regarding '
 'his education.  Daquelle still needs to pass the Math MCAS so he asked for a '
 'calculator to be added to his accommodations.  Daquelle signed the Age of '
 'Majority paperwork, stating that he would like to share educational '
 'responsibilities with his mother after he turns 18.  Judy will also make a '
 '688 referral and send the pa

In [55]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

datanpb_words = list(sent_to_words(datanpb))
datapb_words = list(sent_to_words(datapb))

datanph_words = list(sent_to_words(datanph))
dataph_words = list(sent_to_words(dataph))

datanpw_words = list(sent_to_words(datanpw))
datapw_words = list(sent_to_words(datapw))

datanpp_words = list(sent_to_words(datanpp))
datapp_words = list(sent_to_words(datapp))

datanpwp_words = list(sent_to_words(datanpwp))
datapwp_words = list(sent_to_words(datapwp))

print(datanpb_words[:1])

[['this', 'writer', 'attended', 'daquelle', 'iep', 'meeting', 'on', 'yesterday', 'judy', 'cook', 'bps', 'sess', 'coordinator', 'was', 'able', 'to', 'speak', 'with', 'his', 'mom', 'prior', 'to', 'the', 'meeting', 'and', 'was', 'given', 'permission', 'to', 'hold', 'the', 'meeting', 'mom', 'was', 'not', 'able', 'to', 'attend', 'because', 'she', 'had', 'to', 'work', 'mom', 'shared', 'her', 'concern', 'that', 'she', 'does', 'not', 'want', 'daquelle', 'to', 'return', 'to', 'mckinley', 'however', 'judy', 'said', 'that', 'he', 'would', 'need', 'to', 'return', 'to', 'mckinley', 'and', 'the', 'folks', 'at', 'mckinley', 'would', 'need', 'to', 'determine', 'that', 'he', 'no', 'longer', 'needs', 'that', 'placement', 'when', 'daquelle', 'returns', 'to', 'the', 'community', 'he', 'would', 'be', 'in', 'the', 'th', 'grade', 'daquelle', 'attended', 'the', 'meeting', 'and', 'he', 'asked', 'appropriate', 'questions', 'regarding', 'his', 'education', 'daquelle', 'still', 'needs', 'to', 'pass', 'the', 'math

In [67]:
# Build the bigram and trigram models

def big_tri(datanp_words,datap_words):
    bigramnp = gensim.models.Phrases(datanp_words, min_count=5, threshold=100) # higher threshold fewer phrases.
    trigramnp = gensim.models.Phrases(bigramnp[datanp_words], threshold=100)  
    
    bigramp = gensim.models.Phrases(datap_words, min_count=5, threshold=100) # higher threshold fewer phrases.
    trigramp = gensim.models.Phrases(bigramp[datap_words], threshold=100)  
    
    # Faster way to get a sentence clubbed as a trigram/bigram
    bigramnp_mod = gensim.models.phrases.Phraser(bigramnp)
    trigramnp_mod = gensim.models.phrases.Phraser(trigramnp)
    
    bigramp_mod = gensim.models.phrases.Phraser(bigramp)
    trigramp_mod = gensim.models.phrases.Phraser(trigramp)
    
    # See trigram example
    print(trigramnp_mod[bigramnp_mod[datanp_words[0]]])
    return bigramnp_mod,bigramp_mod,trigramnp_mod,trigramp_mod

bigramnpb_mod,bigrampb_mod,trigramnpb_mod,trigrampb_mod=big_tri(datanpb_words,datapb_words)
bigramnpw_mod,bigrampw_mod,trigramnpw_mod,trigrampw_mod=big_tri(datanpw_words,datapw_words)
bigramnph_mod,bigramph_mod,trigramnph_mod,trigramph_mod=big_tri(datanph_words,dataph_words)
bigramnpp_mod,bigrampp_mod,trigramnpp_mod,trigrampp_mod=big_tri(datanpp_words,datapp_words)
bigramnpwp_mod,bigrampwp_mod,trigramnpwp_mod,trigrampwp_mod=big_tri(datanpwp_words,datapwp_words)

['this', 'writer', 'attended', 'daquelle', 'iep', 'meeting', 'on', 'yesterday', 'judy', 'cook', 'bps', 'sess', 'coordinator', 'was', 'able', 'to', 'speak', 'with', 'his', 'mom', 'prior', 'to', 'the', 'meeting', 'and', 'was', 'given', 'permission', 'to', 'hold', 'the', 'meeting', 'mom', 'was', 'not', 'able', 'to', 'attend', 'because', 'she', 'had', 'to', 'work', 'mom', 'shared', 'her', 'concern', 'that', 'she', 'does', 'not', 'want', 'daquelle', 'to', 'return', 'to', 'mckinley', 'however', 'judy', 'said', 'that', 'he', 'would', 'need', 'to', 'return', 'to', 'mckinley', 'and', 'the', 'folks', 'at', 'mckinley', 'would', 'need', 'to', 'determine', 'that', 'he', 'no', 'longer', 'needs', 'that', 'placement', 'when', 'daquelle', 'returns', 'to', 'the', 'community', 'he', 'would', 'be', 'in', 'the', 'th_grade', 'daquelle', 'attended', 'the', 'meeting', 'and', 'he', 'asked', 'appropriate', 'questions', 'regarding', 'his', 'education', 'daquelle', 'still', 'needs', 'to', 'pass', 'the', 'math_mca

In [75]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts,pnp,bigramnp_mod,bigramp_mod):
    if pnp=='np':
        return [bigramnp_mod[doc] for doc in texts]
    else:
        return [bigramp_mod[doc] for doc in texts]

def make_trigrams(texts,pnp):
    if pnp=='np':
        return [trigramnp_mod[bigramnp_mod[doc]] for doc in texts]
    else:
        return [trigramp_mod[bigramp_mod[doc]] for doc in texts]

def lemmatization(texts,nlp,allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [76]:
def get_lem(datanp_words,datap_words,bigramnp_mod,bigramp_mod):
    # Remove Stop Words
    datanp_words_nostops = remove_stopwords(datanp_words)
    datap_words_nostops = remove_stopwords(datap_words)
    
    # Form Bigrams
    datanp_words_bigrams = make_bigrams(datanp_words_nostops,'np',bigramnp_mod,bigramp_mod)
    datap_words_bigrams = make_bigrams(datap_words_nostops,'p',bigramnp_mod,bigramp_mod)
    
    # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
    # python3 -m spacy download en
    nlp = spacy.load('en',disable=['parser', 'ner'])
    
    # Do lemmatization keeping only noun, adj, vb, adv
    datanp_lemmatized = lemmatization(datanp_words_bigrams, nlp, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    datap_lemmatized = lemmatization(datap_words_bigrams, nlp, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
    
    print(datanp_lemmatized[:1])
    return datanp_lemmatized,datap_lemmatized

datanpb_lemmatized,datapb_lemmatized=get_lem(datanpb_words,datapb_words,bigramnpb_mod,bigrampb_mod)
datanph_lemmatized,dataph_lemmatized=get_lem(datanph_words,dataph_words,bigramnph_mod,bigramph_mod)
datanpw_lemmatized,datapw_lemmatized=get_lem(datanpw_words,datapw_words,bigramnpw_mod,bigrampw_mod)
datanpp_lemmatized,datapp_lemmatized=get_lem(datanpp_words,datapp_words,bigramnpp_mod,bigrampp_mod)
datanpwp_lemmatized,datapwp_lemmatized=get_lem(datanpwp_words,datapwp_words,bigramnpwp_mod,bigrampwp_mod)

[['writer', 'attend', 'sess', 'coordinator', 'speak', 'mom', 'prior', 'meet', 'give', 'permission', 'hold', 'meeting', 'mom', 'attend', 'work', 'mom', 'share', 'concern', 'want', 'however', 'say', 'folk', 'nee', 'long', 'need', 'placement', 'daquelle', 'return', 'attend', 'meeting', 'ask', 'appropriate', 'question', 'regard', 'education', 'daquelle', 'need', 'ask', 'calculator', 'add', 'accommodation', 'sign', 'age', 'majority', 'paperwork', 'state', 'share', 'educational', 'mother', 'turn', 'referral', 'send', 'paperwork', 'mom', 'sign', 'mom', 'give', 'need', 'parent', 'center', 'change', 'system']]
[['speak', 'today', 'explain', 'time', 'time', 'assignment', 'concurrent', 'original', 'time', 'assignment', 'discharge', 'really', 'really', 'ask', 'early', 'discharge', 'hard']]
[['mother', 'go', 'criminal', 'remorse', 'involvement', 'tin', 'incident', 'encourage', 'staff', 'issue', 'escalate', 'serious', 'issue', 'work', 'repair']]
[['receive', 'call', 'let', 'teaching', 'roommate', 'r

In [77]:
# Create Dictionary
id2wordnpb = corpora.Dictionary(datanpb_lemmatized)
id2wordpb = corpora.Dictionary(datapb_lemmatized)

id2wordnph = corpora.Dictionary(datanph_lemmatized)
id2wordph = corpora.Dictionary(dataph_lemmatized)

id2wordnpw = corpora.Dictionary(datanpw_lemmatized)
id2wordpw = corpora.Dictionary(datapw_lemmatized)

id2wordnpp = corpora.Dictionary(datanpp_lemmatized)
id2wordpp = corpora.Dictionary(datapp_lemmatized)

id2wordnpwp = corpora.Dictionary(datanpwp_lemmatized)
id2wordpwp = corpora.Dictionary(datapwp_lemmatized)

# Create Corpus
textsnpb = datanpb_lemmatized
textspb = datapb_lemmatized

textsnph = datanph_lemmatized
textsph = dataph_lemmatized

textsnpw = datanpw_lemmatized
textspw = datapw_lemmatized

textsnpp = datanpp_lemmatized
textspp = datapp_lemmatized

textsnpwp = datanpwp_lemmatized
textspwp = datapwp_lemmatized

# Term Document Frequency
corpusnpb = [id2wordnpb.doc2bow(text) for text in textsnpb]
corpuspb = [id2wordpb.doc2bow(text) for text in textspb]

corpusnph = [id2wordnph.doc2bow(text) for text in textsnph]
corpusph = [id2wordph.doc2bow(text) for text in textsph]

corpusnpw = [id2wordnpw.doc2bow(text) for text in textsnpw]
corpuspw = [id2wordpw.doc2bow(text) for text in textspw]

corpusnpp = [id2wordnpp.doc2bow(text) for text in textsnpp]
corpuspp = [id2wordpp.doc2bow(text) for text in textspp]

corpusnpwp = [id2wordnpwp.doc2bow(text) for text in textsnpwp]
corpuspwp = [id2wordpwp.doc2bow(text) for text in textspwp]
# View
print(corpuspb[:1])
print(corpusph[:1])
print(corpuspw[:1])
print(corpuspp[:1])
print(corpuspwp[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 2), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1)]]
[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 2), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 2), (20, 1), (21, 2), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 8)]]
[[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 2), (12, 1), (13, 2), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1)]]
[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 2), (8, 2), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1), (16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 8)]]
[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (

In [78]:
#Black
lda_modelnpb = gensim.models.ldamodel.LdaModel(corpus=corpusnpb,
                                           id2word=id2wordnpb,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                            per_word_topics=True)
lda_modelpb = gensim.models.ldamodel.LdaModel(corpus=corpuspb,
                                           id2word=id2wordpb,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
#Hispanic
lda_modelnph = gensim.models.ldamodel.LdaModel(corpus=corpusnph,
                                           id2word=id2wordnph,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                            per_word_topics=True)
lda_modelph = gensim.models.ldamodel.LdaModel(corpus=corpusph,
                                           id2word=id2wordph,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
#White
lda_modelnpw = gensim.models.ldamodel.LdaModel(corpus=corpusnpw,
                                           id2word=id2wordnpw,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                            per_word_topics=True)
lda_modelpw = gensim.models.ldamodel.LdaModel(corpus=corpuspw,
                                           id2word=id2wordpw,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
#person
lda_modelnpp = gensim.models.ldamodel.LdaModel(corpus=corpusnpp,
                                           id2word=id2wordnpp,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                            per_word_topics=True)
lda_modelpp = gensim.models.ldamodel.LdaModel(corpus=corpuspp,
                                           id2word=id2wordpp,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
#Weapons
lda_modelnpwp = gensim.models.ldamodel.LdaModel(corpus=corpusnpwp,
                                           id2word=id2wordnpwp,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                            per_word_topics=True)
lda_modelpwp = gensim.models.ldamodel.LdaModel(corpus=corpuspwp,
                                           id2word=id2wordpwp,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [79]:
print('non-par black: ')
print(lda_modelnpb.print_topics())
doc_ldanpb = lda_modelnpb[corpusnpb]

print('par black: ')
print(lda_modelpb.print_topics())
doc_ldapb = lda_modelpb[corpuspb]

non-par black: 
[(0, '0.214*"tell" + 0.180*"office" + 0.179*"phone" + 0.149*"receive" + 0.109*"come" + 0.061*"information" + 0.026*"change" + 0.026*"arrive" + 0.015*"monitor" + 0.006*"around"'), (1, '0.133*"home" + 0.104*"call" + 0.097*"report" + 0.063*"speak" + 0.060*"visit" + 0.059*"mother" + 0.059*"meet" + 0.054*"parent" + 0.053*"inform" + 0.045*"time"'), (2, '0.326*"attempt" + 0.241*"regard" + 0.070*"email" + 0.047*"fine" + 0.047*"month" + 0.043*"assault" + 0.041*"peer" + 0.039*"happen" + 0.031*"lose" + 0.022*"extension"'), (3, '0.102*"go" + 0.096*"state" + 0.072*"work" + 0.069*"ask" + 0.065*"say" + 0.049*"need" + 0.046*"concern" + 0.042*"day" + 0.040*"back" + 0.036*"want"'), (4, '0.310*"check" + 0.281*"program" + 0.092*"present" + 0.088*"pass" + 0.060*"test" + 0.037*"discharge" + 0.035*"clinical" + 0.019*"director" + 0.018*"track" + 0.015*"assignment"'), (5, '0.385*"transport" + 0.135*"request" + 0.111*"later" + 0.099*"staffing" + 0.055*"see" + 0.049*"allow" + 0.041*"begin" + 0.02

In [80]:
print('non-par hispanic: ')
print(lda_modelnph.print_topics())
doc_ldanph = lda_modelnph[corpusnph]

print('par hispanic: ')
print(lda_modelph.print_topics())
doc_ldaph = lda_modelph[corpusph]

non-par hispanic: 
[(0, '0.270*"update" + 0.219*"hold" + 0.120*"incident" + 0.085*"bail" + 0.069*"note" + 0.061*"mention" + 0.035*"close" + 0.029*"necessary" + 0.028*"apprehend" + 0.012*"throw"'), (1, '0.421*"speak" + 0.176*"family" + 0.077*"look" + 0.056*"friend" + 0.047*"step" + 0.045*"police" + 0.038*"father" + 0.023*"happen" + 0.017*"walk" + 0.015*"suggest"'), (2, '0.156*"go" + 0.117*"state" + 0.083*"say" + 0.076*"week" + 0.068*"ask" + 0.060*"need" + 0.055*"tell" + 0.053*"get" + 0.049*"want" + 0.049*"next"'), (3, '0.270*"mother" + 0.182*"meeting" + 0.144*"pick" + 0.086*"take" + 0.065*"bring" + 0.042*"director" + 0.041*"participate" + 0.034*"clinical" + 0.028*"try" + 0.016*"unable"'), (4, '0.415*"call" + 0.144*"pass" + 0.107*"phone" + 0.098*"back" + 0.046*"girlfriend" + 0.043*"start" + 0.042*"remind" + 0.017*"district" + 0.016*"list" + 0.015*"already"'), (5, '0.199*"receive" + 0.151*"schedule" + 0.130*"set" + 0.109*"email" + 0.101*"test" + 0.095*"request" + 0.054*"decision" + 0.038*

In [81]:
print('non-par white: ')
print(lda_modelnpw.print_topics())
doc_ldanpw = lda_modelnpw[corpusnpw]

print('par white: ')
print(lda_modelpw.print_topics())
doc_ldapw = lda_modelpw[corpuspw]

non-par white: 
[(0, '0.173*"program" + 0.143*"discuss" + 0.126*"contact" + 0.098*"plan" + 0.087*"regard" + 0.074*"student" + 0.052*"agree" + 0.030*"support" + 0.027*"hold" + 0.019*"offer"'), (1, '0.000*"reingage" + 0.000*"imagine" + 0.000*"feed" + 0.000*"offense_cycle" + 0.000*"katie" + 0.000*"soberiety" + 0.000*"confidence" + 0.000*"denver" + 0.000*"fan" + 0.000*"improved_greatly"'), (2, '0.399*"need" + 0.323*"want" + 0.177*"thing" + 0.020*"info" + 0.010*"opportunity" + 0.008*"introduce" + 0.007*"store" + 0.000*"writer" + 0.000*"jake" + 0.000*"youth"'), (3, '0.307*"client" + 0.294*"home" + 0.097*"visit" + 0.078*"parent" + 0.064*"community" + 0.061*"appointment" + 0.027*"see" + 0.020*"well" + 0.017*"event" + 0.011*"assign"'), (4, '0.618*"tomorrow" + 0.220*"confirm" + 0.000*"writer" + 0.000*"remind" + 0.000*"therapy" + 0.000*"screening" + 0.000*"screen" + 0.000*"youth" + 0.000*"inform" + 0.000*"cody"'), (5, '0.173*"time" + 0.128*"day" + 0.109*"attend" + 0.098*"make" + 0.096*"return" + 

In [82]:
print('non-par person: ')
print(lda_modelnpp.print_topics())
doc_ldanpp = lda_modelnpp[corpusnpp]

print('par person: ')
print(lda_modelpp.print_topics())
doc_ldapp = lda_modelpp[corpuspp]

non-par person: 
[(0, '0.517*"go" + 0.262*"say" + 0.119*"take" + 0.023*"prior" + 0.019*"already" + 0.017*"year" + 0.011*"person" + 0.008*"successful" + 0.000*"report" + 0.000*"writer"'), (1, '0.507*"class" + 0.098*"include" + 0.088*"name" + 0.060*"gang" + 0.050*"write" + 0.028*"other" + 0.025*"classroom" + 0.018*"apparently" + 0.000*"report" + 0.000*"writer"'), (2, '0.392*"meeting" + 0.196*"case" + 0.150*"treatment" + 0.079*"email" + 0.052*"well" + 0.048*"forward" + 0.023*"communication" + 0.015*"region" + 0.007*"cover" + 0.000*"report"'), (3, '0.863*"job" + 0.000*"report" + 0.000*"writer" + 0.000*"office" + 0.000*"application" + 0.000*"search" + 0.000*"thing" + 0.000*"finn" + 0.000*"community" + 0.000*"gp"'), (4, '0.756*"home" + 0.184*"come" + 0.021*"baby" + 0.007*"town" + 0.000*"report" + 0.000*"writer" + 0.000*"office" + 0.000*"gp" + 0.000*"pass" + 0.000*"jake"'), (5, '0.261*"schedule" + 0.157*"request" + 0.075*"d" + 0.073*"dad" + 0.061*"arrive" + 0.057*"door" + 0.037*"happy" + 0.03

In [83]:
print('non-par weapons: ')
print(lda_modelnpwp.print_topics())
doc_ldanpwp = lda_modelnpwp[corpusnpwp]

print('par weapons: ')
print(lda_modelpwp.print_topics())
doc_ldapwp = lda_modelpwp[corpuspwp]

non-par weapons: 
[(0, '0.217*"update" + 0.172*"sign" + 0.123*"release" + 0.083*"curfew" + 0.077*"bail" + 0.059*"violation" + 0.059*"warrant" + 0.041*"close" + 0.029*"note" + 0.025*"accord"'), (1, '0.595*"time" + 0.108*"service" + 0.091*"early" + 0.054*"really" + 0.045*"hard" + 0.027*"assignment" + 0.005*"around" + 0.000*"nichola" + 0.000*"caseworker" + 0.000*"behavior"'), (2, '0.349*"information" + 0.212*"confirm" + 0.103*"see" + 0.074*"copy" + 0.067*"happen" + 0.018*"clear" + 0.000*"nichola" + 0.000*"unit" + 0.000*"antoine" + 0.000*"kenny"'), (3, '0.290*"pass" + 0.120*"hold" + 0.091*"email" + 0.089*"month" + 0.072*"set" + 0.062*"discharge" + 0.055*"submit" + 0.047*"soon" + 0.042*"district" + 0.031*"past"'), (4, '0.256*"placement" + 0.115*"possible" + 0.108*"obtain" + 0.090*"notify" + 0.082*"status" + 0.067*"referral" + 0.043*"suggest" + 0.032*"walk" + 0.018*"self" + 0.016*"likely"'), (5, '0.464*"next" + 0.347*"schedule" + 0.077*"vacation" + 0.000*"nichola" + 0.000*"caseworker" + 0.00

In [89]:
pyLDAvis.enable_notebook()
visnpb = pyLDAvis.gensim.prepare(lda_modelnpb, corpusnpb, id2wordnpb)

In [90]:
visnpb

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.297086 -0.329631       1        1  20.864567
3      0.244052  0.355471       2        1  17.316753
12     0.046241  0.004727       3        1   6.081900
6      0.089059  0.013591       4        1   6.008866
15     0.057599  0.006841       5        1   5.268576
14     0.086536  0.012770       6        1   4.968014
4      0.030710  0.002800       7        1   4.796280
16     0.072290  0.009945       8        1   4.545770
0      0.028386  0.002483       9        1   4.469378
19    -0.042910 -0.004867      10        1   3.169780
8     -0.077510 -0.007068      11        1   2.796712
18    -0.024188 -0.003390      12        1   2.635891
9     -0.086646 -0.007486      13        1   2.486835
5     -0.070444 -0.006663      14        1   2.335529
13    -0.105783 -0.008280      15        1   2.320332
2     -0.078007 -0.007058      16        1   2.117405
11    -0.120526 -0.008642      17        1   2.101298
10    -0.118210 -0.008583      18        1   1.958731
17    -0.108612 -0.008341      19        1   1.913141
7     -0.119123 -0.008619      20        1   1.844240, topic_info=            Term          Freq         Total Category  logprob  loglift
51        client  18416.000000  18416.000000  Default  30.0000  30.0000
46        writer  11017.000000  11017.000000  Default  29.0000  29.0000
90         youth   9438.000000   9438.000000  Default  28.0000  28.0000
236       school   8811.000000   8811.000000  Default  27.0000  27.0000
63         check   7034.000000   7034.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
684          dys      0.081676      1.536815  Topic20 -11.5802   1.0584
1530      obtain      0.081673      1.536695  Topic20 -11.5802   1.0584
1436     student      0.081679      1.537080  Topic20 -11.5801   1.0583
615         drop      0.081678      1.537056  Topic20 -11.5801   1.0583
849   compliance      0.081678      1.537163  Topic20 -11.5801   1.0582

[844 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
510       6  0.994570          accept
285       8  0.918528          access
110       3  0.972381  accountability
268       1  0.995875          action
528      19  0.997072        activity
...     ...       ...             ...
250      10  0.990289           write
46        8  0.999845          writer
60       16  0.992853             yet
431      12  0.990012           young
90        6  0.999810           youth

[360 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 13, 7, 16, 15, 5, 17, 1, 20, 9, 19, 10, 6, 14, 3, 12, 11, 18, 8])

In [91]:
pyLDAvis.enable_notebook()
vispb = pyLDAvis.gensim.prepare(lda_modelpb, corpuspb, id2wordpb)

In [92]:
vispb

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
16    -0.427114  0.190339       1        1  17.038170
9     -0.082935 -0.298837       2        1   6.830860
17    -0.081702 -0.283871       3        1   6.439963
13    -0.039176 -0.057633       4        1   5.500918
2     -0.034847 -0.047400       5        1   5.197710
3      0.030367  0.030022       6        1   5.017396
0     -0.012081 -0.008703       7        1   5.004693
19     0.006039  0.011460       8        1   4.948033
1      0.020868  0.023123       9        1   4.480402
18     0.022904  0.024196      10        1   4.386823
10     0.046586  0.038712      11        1   4.223125
6      0.027708  0.027824      12        1   4.162913
14     0.033123  0.030928      13        1   4.116369
4      0.058965  0.044105      14        1   3.835224
15     0.029692  0.029177      15        1   3.813393
5      0.051049  0.040256      16        1   3.716966
11     0.080495  0.050753      17        1   3.193617
7      0.076551  0.049323      18        1   3.085758
8      0.085542  0.051892      19        1   3.010952
12     0.107968  0.054334      20        1   1.996714, topic_info=         Term          Freq         Total Category  logprob  loglift
40       home  18612.000000  18612.000000  Default  30.0000  30.0000
43       call  21528.000000  21528.000000  Default  29.0000  29.0000
75     writer  19425.000000  19425.000000  Default  28.0000  28.0000
96     mother  14171.000000  14171.000000  Default  27.0000  27.0000
230     today  11524.000000  11524.000000  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
568   explain      0.084494      1.651844  Topic20 -12.1777   0.9407
683     thing      0.084493      1.651807  Topic20 -12.1777   0.9407
566      test      0.084492      1.651747  Topic20 -12.1777   0.9407
476      find      0.084492      1.651676  Topic20 -12.1777   0.9408
1405       gp      0.084492      1.651803  Topic20 -12.1777   0.9407

[910 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
397       5  0.974186     academy
319      14  0.998680      accept
298      17  0.990628   accompany
191       6  0.985960    actively
266      12  0.996885  additional
...     ...       ...         ...
167       6  0.997966       write
75        5  0.999934      writer
210       6  0.998667        year
285       9  0.997404         yet
102       5  0.999866       youth

[308 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[17, 10, 18, 14, 3, 4, 1, 20, 2, 19, 11, 7, 15, 5, 16, 6, 12, 8, 9, 13])

In [93]:
#H,np
pyLDAvis.enable_notebook()
visnph = pyLDAvis.gensim.prepare(lda_modelnph, corpusnph, id2wordnph)

In [94]:
visnph

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2      0.195103 -0.368482       1        1  12.875504
14     0.183782  0.307196       2        1  12.595958
11     0.165879  0.070843       3        1  11.980088
7      0.110106  0.006284       4        1   6.660937
3      0.086933  0.002981       5        1   6.083454
4      0.099015  0.003970       6        1   5.989941
17     0.062244  0.001173       7        1   5.297014
19     0.039532 -0.000043       8        1   5.079378
9      0.043838  0.000090       9        1   4.876352
13    -0.017900 -0.001847      10        1   3.933023
5      0.010313 -0.001146      11        1   3.927122
1     -0.004358 -0.001541      12        1   3.899750
6     -0.028361 -0.002041      13        1   3.397325
18    -0.100508 -0.002798      14        1   3.249474
15    -0.054555 -0.002394      15        1   2.980630
12    -0.097176 -0.002791      16        1   2.393963
16    -0.168938 -0.003016      17        1   1.657853
10    -0.152448 -0.002998      18        1   1.641460
0     -0.142310 -0.002997      19        1   1.463732
8     -0.230189 -0.000443      20        1   0.017030, topic_info=            Term         Freq         Total Category  logprob  loglift
109         call  17281.00000  17281.000000  Default  30.0000  30.0000
414       client  17532.00000  17532.000000  Default  29.0000  29.0000
14          home  14105.00000  14105.000000  Default  28.0000  28.0000
11         today  11596.00000  11596.000000  Default  27.0000  27.0000
9          speak  11402.00000  11402.000000  Default  26.0000  26.0000
...          ...          ...           ...      ...      ...      ...
3590  grandchild      0.01106      1.557903  Topic20  -9.2783   3.7302
3589    showtime      0.01106      1.557903  Topic20  -9.2783   3.7302
3588    exercise      0.01106      1.557910  Topic20  -9.2783   3.7302
3587     urgency      0.01106      1.557903  Topic20  -9.2783   3.7302
3586    agreeing      0.01106      1.557903  Topic20  -9.2783   3.7302

[861 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
468       2  0.995743  acknowledge
263      13  0.995351       action
63        3  0.997650     activity
442       6  0.997869          add
433       1  0.994827        admit
...     ...       ...          ...
271      13  0.994171        write
208       4  0.999798       writer
237       8  0.998128         year
272      13  0.997678          yet
200       7  0.999748        youth

[367 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 15, 12, 8, 4, 5, 18, 20, 10, 14, 6, 2, 7, 19, 16, 13, 17, 11, 1, 9])

In [95]:
#H,p
pyLDAvis.enable_notebook()
visph = pyLDAvis.gensim.prepare(lda_modelph, corpusph, id2wordph)

In [96]:
visph

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.226295  0.358283       1        1  16.282230
0      0.175526 -0.320749       2        1  14.238083
6      0.158145 -0.058107       3        1   8.543772
13     0.106234 -0.020165       4        1   6.820283
19     0.105748 -0.019110       5        1   6.158092
4      0.087619 -0.012774       6        1   5.415447
17     0.097427 -0.015685       7        1   5.280434
18     0.035914 -0.001514       8        1   5.225840
1      0.041901 -0.002441       9        1   4.441852
2     -0.006411  0.003835      10        1   3.376561
15    -0.082015  0.009303      11        1   3.206872
9     -0.067783  0.008613      12        1   3.109345
10    -0.089552  0.009528      13        1   2.878187
7     -0.096379  0.009689      14        1   2.807064
3     -0.099632  0.009865      15        1   2.770253
14    -0.069207  0.008361      16        1   2.671628
8     -0.066459  0.008301      17        1   2.559898
11    -0.112899  0.010227      18        1   2.215086
16    -0.127074  0.010695      19        1   1.983683
12    -0.217399  0.003845      20        1   0.015387, topic_info=             Term          Freq         Total Category  logprob  loglift
78           call  33071.000000  33071.000000  Default  30.0000  30.0000
322          home  28251.000000  28251.000000  Default  29.0000  29.0000
312        school  20866.000000  20866.000000  Default  28.0000  28.0000
116            go  29967.000000  29967.000000  Default  27.0000  27.0000
234         today  29803.000000  29803.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
4893  reconstruct      0.013252      1.706812  Topic20  -9.5975   3.9211
4892       podium      0.013252      1.706812  Topic20  -9.5975   3.9211
4917        shove      0.013252      1.706823  Topic20  -9.5975   3.9211
4919        danny      0.013252      1.706826  Topic20  -9.5975   3.9211
4890        datum      0.013252      1.706812  Topic20  -9.5975   3.9211

[884 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
388       6  0.998160    accept
367       7  0.999084    accord
235       4  0.995570   account
265      12  0.999006  activity
151       1  0.999527     agree
...     ...       ...       ...
230       1  0.999921      work
65       14  0.999894    writer
415       6  0.999138      year
278       6  0.987005      ymca
28       15  0.999779     youth

[299 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 1, 7, 14, 20, 5, 18, 19, 2, 3, 16, 10, 11, 8, 4, 15, 9, 12, 17, 13])

In [97]:
#Wh,np
pyLDAvis.enable_notebook()
visnpw = pyLDAvis.gensim.prepare(lda_modelnpw, corpusnpw, id2wordnpw)

In [98]:
visnpw

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
10    -0.183166  0.416100       1        1  15.525807
19    -0.150247 -0.206654       2        1  12.406981
3     -0.147705 -0.115186       3        1   9.398955
0     -0.118014 -0.045992       4        1   8.885167
5     -0.108121 -0.035213       5        1   7.877396
15    -0.087846 -0.021849       6        1   5.917212
6     -0.083796 -0.019802       7        1   5.688759
17    -0.092770 -0.025782       8        1   5.296217
13    -0.059382 -0.011283       9        1   4.792356
11    -0.010504  0.000231      10        1   4.025693
14    -0.022300 -0.001261      11        1   3.782055
8     -0.005348  0.001020      12        1   3.474415
18    -0.009990 -0.000117      13        1   3.384320
2      0.016119  0.003455      14        1   2.781457
12     0.027902  0.005467      15        1   2.410059
7      0.066786  0.008457      16        1   2.235008
4      0.193510  0.013348      17        1   0.884020
16     0.231596  0.013379      18        1   0.708495
9      0.251310  0.012999      19        1   0.505407
1      0.291966  0.008681      20        1   0.020214, topic_info=            Term          Freq         Total Category  logprob  loglift
37        report  20423.000000  20423.000000  Default  30.0000  30.0000
41        client  14722.000000  14722.000000  Default  29.0000  29.0000
128         home  14094.000000  14094.000000  Default  28.0000  28.0000
3             go  14918.000000  14918.000000  Default  27.0000  27.0000
21         today  10168.000000  10168.000000  Default  26.0000  26.0000
...          ...           ...           ...      ...      ...      ...
3214  infraction      0.010765      1.588344  Topic20  -9.1685   3.5125
3213      adhere      0.010765      1.588333  Topic20  -9.1685   3.5125
3212  perfection      0.010765      1.588333  Topic20  -9.1685   3.5125
3211       would      0.010765      1.588333  Topic20  -9.1685   3.5125
3210        wash      0.010765      1.588334  Topic20  -9.1685   3.5125

[906 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
154       4  0.992796     access
224       6  0.978876  accompany
126      16  0.990737     active
172       7  0.997561     advise
249      15  0.980139     affect
...     ...       ...        ...
344       3  0.998336       well
216      10  0.978614      whole
13        9  0.999793       work
23        2  0.995518       year
272      15  0.987727      young

[282 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[11, 20, 4, 1, 6, 16, 7, 18, 14, 12, 15, 9, 19, 3, 13, 8, 5, 17, 10, 2])

In [99]:
#Wh,p
pyLDAvis.enable_notebook()
vispw = pyLDAvis.gensim.prepare(lda_modelpw, corpuspw, id2wordpw)

In [100]:
vispw

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
19     0.256611 -0.368103       1        1  11.033422
3      0.170465  0.274622       2        1   8.740760
2      0.118942  0.084039       3        1   8.342552
14     0.114854  0.059336       4        1   6.996800
1      0.108269  0.062400       5        1   6.497896
11     0.121762  0.063690       6        1   6.378067
0      0.005001 -0.003158       7        1   5.235863
4      0.018197  0.001490       8        1   4.859970
8     -0.000306 -0.003650       9        1   4.676729
5     -0.032802 -0.010902      10        1   4.569157
13    -0.024365 -0.008888      11        1   4.438159
18    -0.009912 -0.005969      12        1   4.405355
15    -0.061924 -0.015520      13        1   4.232070
16    -0.060043 -0.014820      14        1   4.027092
7     -0.097894 -0.020713      15        1   3.886174
12    -0.064377 -0.015096      16        1   3.362101
10    -0.095637 -0.019251      17        1   3.324256
9     -0.123424 -0.021679      18        1   2.627229
17    -0.154356 -0.024213      19        1   2.346667
6     -0.189059 -0.013614      20        1   0.019688, topic_info=              Term          Freq         Total Category  logprob  loglift
33            call  18813.000000  18813.000000  Default  30.0000   30.000
52            home  16017.000000  16017.000000  Default  29.0000   29.000
58          report  19175.000000  19175.000000  Default  28.0000   28.000
3           client  10665.000000  10665.000000  Default  27.0000   27.000
335           work  11188.000000  11188.000000  Default  26.0000   26.000
...            ...           ...           ...      ...      ...      ...
3609     summonsed      0.011598      1.750668  Topic20  -9.2832    3.516
3608          cuff      0.011598      1.750684  Topic20  -9.2832    3.516
3607      merrimac      0.011598      1.750673  Topic20  -9.2832    3.516
3606  homelessness      0.011598      1.750665  Topic20  -9.2832    3.516
3605       abandon      0.011598      1.750667  Topic20  -9.2832    3.516

[897 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
224      15  0.997251    activity
187       3  0.994383  additional
228       4  0.998394   afternoon
328       5  0.991078  afterwards
298       3  0.995717         age
...     ...       ...         ...
226      15  0.999102      worker
345      14  0.997244     working
332       3  0.996426       write
296      13  0.999811      writer
242       7  0.999706       youth

[289 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[20, 4, 3, 15, 2, 12, 1, 5, 9, 6, 14, 19, 16, 17, 8, 13, 11, 10, 18, 7])

In [101]:
#P,np
pyLDAvis.enable_notebook()
visnpp = pyLDAvis.gensim.prepare(lda_modelnpp, corpusnpp, id2wordnpp)

In [102]:
visnpp

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
8      0.219737 -0.433812       1        1  37.482632
13     0.130663  0.091645       2        1   6.989694
11     0.119950  0.084876       3        1   6.273664
15     0.082205  0.057250       4        1   5.019105
0      0.109692  0.080825       5        1   4.930822
16     0.130267  0.103311       6        1   4.766540
9      0.056702  0.039463       7        1   4.367795
19     0.043209  0.034102       8        1   3.920116
17     0.039993  0.032031       9        1   3.781932
18     0.043759  0.033089      10        1   3.688553
2      0.026506  0.024408      11        1   3.492834
4      0.051818  0.039368      12        1   3.439130
5     -0.022268  0.001735      13        1   3.005000
7     -0.021053  0.003518      14        1   2.501111
12    -0.047130 -0.005371      15        1   2.142312
6     -0.137906 -0.028966      16        1   1.355768
10    -0.181536 -0.036873      17        1   1.103728
3     -0.195996 -0.038998      18        1   0.882363
1     -0.185951 -0.037730      19        1   0.839036
14    -0.262659 -0.043873      20        1   0.017875, topic_info=           Term          Freq         Total Category  logprob  loglift
164      client  31568.000000  31568.000000  Default  30.0000  30.0000
0          call  28573.000000  28573.000000  Default  29.0000  29.0000
176        home  26391.000000  26391.000000  Default  28.0000  28.0000
12           go  25903.000000  25903.000000  Default  27.0000  27.0000
132       check  16301.000000  16301.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
4532     emerge      0.013366      1.778723  Topic20  -9.5161   3.7385
4534        myr      0.013366      1.778745  Topic20  -9.5161   3.7385
4242     untrue      0.013366      1.778728  Topic20  -9.5161   3.7385
4535  years_old      0.013366      1.778726  Topic20  -9.5161   3.7385
4558  stadolink      0.013366      1.778723  Topic20  -9.5161   3.7385

[947 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
393       7  0.998533     advise
256      15  0.998233  afternoon
97       13  0.997412        age
191       5  0.998390    already
46       14  0.999483     answer
...     ...       ...        ...
215       1  0.999933       work
62       19  0.995153      write
127       5  0.997618       year
391       7  0.997129        yet
68       10  0.999859      youth

[218 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[9, 14, 12, 16, 1, 17, 10, 20, 18, 19, 3, 5, 6, 8, 13, 7, 11, 4, 2, 15])

In [103]:
#P,p
pyLDAvis.enable_notebook()
vispp = pyLDAvis.gensim.prepare(lda_modelpp, corpuspp, id2wordpp)

In [104]:
vispp

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
17    -0.425509  0.194252       1        1  18.651510
1     -0.079832 -0.348064       2        1   7.281988
8     -0.062325 -0.172468       3        1   6.242222
7     -0.051771 -0.115101       4        1   6.215360
15    -0.018336 -0.022943       5        1   6.150808
14    -0.025353 -0.035651       6        1   5.626916
11    -0.036716 -0.063442       7        1   5.191569
0      0.027751  0.031157       8        1   4.399250
13     0.023408  0.027558       9        1   4.282708
16     0.011004  0.015864      10        1   4.248107
4      0.029518  0.031832      11        1   3.787030
18     0.053606  0.046344      12        1   3.743532
19     0.059345  0.049218      13        1   3.663958
3      0.052588  0.045672      14        1   3.638128
5      0.056347  0.047483      15        1   3.582674
2      0.047344  0.042674      16        1   3.148415
6      0.075768  0.055423      17        1   2.990077
10     0.064111  0.050939      18        1   2.890270
9      0.085151  0.057993      19        1   2.474917
12     0.113901  0.061258      20        1   1.790561, topic_info=          Term          Freq         Total Category  logprob  loglift
335     client  37266.000000  37266.000000  Default  30.0000  30.0000
77        call  36961.000000  36961.000000  Default  29.0000  29.0000
321       home  32001.000000  32001.000000  Default  28.0000  28.0000
58      report  37602.000000  37602.000000  Default  27.0000  27.0000
64      writer  28550.000000  28550.000000  Default  26.0000  26.0000
...        ...           ...           ...      ...      ...      ...
880      begin      0.088673      1.651485  Topic20 -12.6063   1.0982
11291   reggie      0.088679      1.651777  Topic20 -12.6062   1.0981
467       hear      0.088677      1.651705  Topic20 -12.6063   1.0981
463       wait      0.088675      1.651618  Topic20 -12.6063   1.0981
572    student      0.088677      1.651831  Topic20 -12.6063   1.0980

[889 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
389       3  0.998873       accept
366       9  0.998297       accord
233       2  0.998134      account
188       8  0.997702  acknowledge
264       4  0.999495     activity
...     ...       ...          ...
64       16  0.999961       writer
416       3  0.999148         year
346       5  0.998624          yet
277      10  0.995798         ymca
28       12  0.999899        youth

[311 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[18, 2, 9, 8, 16, 15, 12, 1, 14, 17, 5, 19, 20, 4, 6, 3, 7, 11, 10, 13])

In [105]:
#Wea,np
pyLDAvis.enable_notebook()
visnpwp = pyLDAvis.gensim.prepare(lda_modelnpwp, corpusnpwp, id2wordnpwp)

In [106]:
visnpwp

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
14     0.270707  0.345805       1        1  22.847719
10     0.259566 -0.340057       2        1  21.460880
12     0.170533 -0.013060       3        1   8.641132
6      0.128856 -0.006717       4        1   7.225362
11     0.098849 -0.004195       5        1   6.602005
16     0.000997  0.000166       6        1   3.581018
17     0.054626 -0.001914       7        1   3.483532
3     -0.021152  0.000659       8        1   2.846798
18    -0.047993  0.001258       9        1   2.790064
7     -0.070980  0.001651      10        1   2.436523
9     -0.032185  0.000883      11        1   2.427626
1     -0.051933  0.001253      12        1   2.316050
8     -0.050906  0.001242      13        1   2.246725
13    -0.055412  0.001310      14        1   2.044789
19    -0.065815  0.001486      15        1   2.014888
0     -0.093871  0.001876      16        1   1.947314
5     -0.104717  0.001961      17        1   1.616262
4     -0.132860  0.002177      18        1   1.304305
15    -0.109958  0.002025      19        1   1.149156
2     -0.146351  0.002191      20        1   1.017840, topic_info=           Term         Freq        Total Category  logprob  loglift
106        call  6742.000000  6742.000000  Default  30.0000  30.0000
14         home  4891.000000  4891.000000  Default  29.0000  29.0000
53      program  5936.000000  5936.000000  Default  28.0000  28.0000
416      client  9936.000000  9936.000000  Default  27.0000  27.0000
10         time  3848.000000  3848.000000  Default  26.0000  26.0000
..          ...          ...          ...      ...      ...      ...
820     however     0.072990     1.427591  Topic20 -10.5692   1.6141
786     conduct     0.072993     1.427914  Topic20 -10.5692   1.6139
879      advise     0.072991     1.427779  Topic20 -10.5692   1.6139
974  caseworker     0.072989     1.427969  Topic20 -10.5692   1.6138
511       house     0.072988     1.427708  Topic20 -10.5692   1.6140

[843 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
425      16  0.989106       accord
178       3  0.985836  accountable
260       6  0.993060       action
58        4  0.992446     actively
59        4  0.993647     activity
...     ...       ...          ...
92        4  0.992295      working
205       1  0.999600       writer
234       2  0.997684         year
267       6  0.993904          yet
197       2  0.999720        youth

[328 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[15, 11, 13, 7, 12, 17, 18, 4, 19, 8, 10, 2, 9, 14, 20, 1, 6, 5, 16, 3])

In [107]:
#Wea,p
pyLDAvis.enable_notebook()
vispwp = pyLDAvis.gensim.prepare(lda_modelpwp, corpuspwp, id2wordpwp)

In [108]:
vispwp

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1     -0.253061  0.353834       1        1  14.362910
3     -0.207686 -0.321297       2        1  13.527664
8     -0.123861 -0.041208       3        1   7.215880
2     -0.075918 -0.016981       4        1   7.149192
12    -0.090144 -0.022876       5        1   6.365324
16    -0.026135 -0.004377       6        1   6.039680
15    -0.089974 -0.023089       7        1   5.293533
18    -0.021214 -0.002973       8        1   4.585317
7     -0.047527 -0.009730       9        1   4.112666
11     0.013988  0.002280      10        1   3.971023
13     0.017529  0.002550      11        1   3.830181
0      0.004411  0.001027      12        1   3.804269
10     0.059496  0.008088      13        1   3.799342
17     0.018702  0.003088      14        1   3.784741
6      0.053978  0.007665      15        1   3.458195
19     0.052929  0.007365      16        1   3.030680
14     0.114345  0.012406      17        1   2.705396
4      0.163025  0.014945      18        1   1.712671
9      0.207190  0.016335      19        1   1.225886
5      0.229927  0.012947      20        1   0.025442, topic_info=          Term         Freq        Total Category  logprob  loglift
119     client  9288.000000  9288.000000  Default  30.0000  30.0000
4         call  8011.000000  8011.000000  Default  29.0000  29.0000
14        home  6914.000000  6914.000000  Default  28.0000  28.0000
18      mother  6422.000000  6422.000000  Default  27.0000  27.0000
75        meet  7709.000000  7709.000000  Default  26.0000  26.0000
...        ...          ...          ...      ...      ...      ...
2623  interact     0.010374     1.715253  Topic20  -8.9653   3.1685
2624       dna     0.010374     1.715244  Topic20  -8.9653   3.1685
2625    leagal     0.010374     1.715229  Topic20  -8.9653   3.1685
2626    revise     0.010374     1.715230  Topic20  -8.9653   3.1685
2599      eare     0.010374     1.715229  Topic20  -8.9653   3.1685

[926 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
282      15  0.971544   accompany
1        13  0.990682  additional
192       2  0.992226         age
246       6  0.999293       agree
113       8  0.997240       allow
...     ...       ...         ...
100       2  0.999824        work
51        5  0.989977     working
163       4  0.991627       write
196       4  0.994049         yet
32        5  0.999792       youth

[236 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 9, 3, 13, 17, 16, 19, 8, 12, 14, 1, 11, 18, 7, 20, 15, 5, 10, 6])